# Playground: RAG with OpenAI & Elasticsearch Python Client
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/playground/openai-elasticsearch-client.ipynb)



In [1]:
!pip install -qU elasticsearch openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.4 requires langchain-core<0.2.0,>=0.1.46, but you have langchain-core 0.2.9 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


## Example Integration code
To adapt this example for your use-case:
- Update your connection details in the `es_client`
- Replace the es_query with the query suggested in Playground
- Replace the index_source_fields with the fields you want to use for context, per index. 
- Update the prompt to reflect the Plaground prompt

In [ ]:
from elasticsearch import Elasticsearch
from openai import OpenAI
from getpass import getpass

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Update the Elasticsearch URL based on your own setup
es_client = Elasticsearch("http://localhost:9200", api_key=ELASTIC_API_KEY)

openai_client = OpenAI(
    api_key="OPEN_AI_API_KEY",
)

# Update the source fields based on your context field options
# This example will be for bm25-index, we use the text field for context
index_source_fields = {"bm25-index": ["text"]}


def get_elasticsearch_results(query):
    # Update the query to match your retriever provided in Playground
    es_query = {
        "retriever": {
            "standard": {"query": {"multi_match": {"query": query, "fields": ["text"]}}}
        },
        "size": 3,
    }

    result = es_client.search(index="bm25-index", body=es_query)
    return result["hits"]["hits"]


def create_openai_prompt(question, results):
    context = ""
    for hit in results:
        inner_hit_path = f"{hit['_index']}.{index_source_fields.get(hit['_index'])[0]}"

        ## For semantic_text matches, we need to extract the text from the inner_hits
        if "inner_hits" in hit and inner_hit_path in hit["inner_hits"]:
            context += "\n --- \n".join(
                inner_hit["_source"]["text"]
                for inner_hit in hit["inner_hits"][inner_hit_path]["hits"]["hits"]
            )
        else:
            source_field = index_source_fields.get(hit["_index"])[0]
            hit_context = hit["_source"][source_field]
            context += f"{hit_context}\n"

    # Update the prompt based on your own requirements
    prompt = f"""
  Instructions:
  
  - You are an assistant for question-answering tasks.
  - Answer questions truthfully and factually using only the information presented.
  - If you don't know the answer, just say that you don't know, don't make up an answer!
  - You must always cite the document where the answer was extracted using inline academic citation style [], using the position.
  - Use markdown format for code examples.
  - You are correct, factual, precise, and reliable.
  

  Context:
  {context}

  Question: {question}
  Answer:
  """

    return prompt


def generate_openai_completion(user_prompt):
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": user_prompt},
        ],
    )

    return response.choices[0].message.content


if __name__ == "__main__":
    question = "what is this?"
    elasticsearch_results = get_elasticsearch_results(question)
    context_prompt = create_openai_prompt(question, elasticsearch_results)
    openai_completion = generate_openai_completion(context_prompt)
    print(openai_completion)